In [1]:
import torch

from tqdm import tqdm
from time import time

from vit import VisionTransformer
from utils import *
from dataloader import *
from decatt import DecattLoss

torch.manual_seed(0)

In [2]:
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)

[0, 1]


# CIFAR10

In [3]:
image_size = 32
patch_size = 4
in_channels = 3
mlp_dim = 512
num_classes = 10
num_layers = 12
dropout = 0.1
batch_size = 256

lr = 1e-3
weight_decay = 1e-3
num_epochs = 50

trainloader, testloader = cifar10_loaders(image_size, batch_size)

Files already downloaded and verified
Files already downloaded and verified


### ViT Baseline

In [ ]:
num_heads = 6
embed_dim = 64 * num_heads

model = VisionTransformer(
    image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout
)
model = torch.nn.DataParallel(model.cuda(), device_ids=device_ids).cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
train("vit_baseline_cifar10", model, criterion, optimizer, num_epochs, trainloader, testloader)

# Best Validation Accuracy: 69.210%
# Time to Max Val Accuracy: 20.132

In [ ]:
print(f"Total num params: {sum(p.numel() for p in model.parameters())}")
# Total num params: 11965642

### DeCAtt Loss

In [4]:
num_heads = 3
embed_dim = 64 * num_heads

model = VisionTransformer(
    image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout
)
model = torch.nn.DataParallel(model.cuda(), device_ids=device_ids).cuda()

criterion = DecattLoss(num_heads)
optimizer1 = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
optimizer2 = torch.optim.Adam(model.module.transformer1.parameters(), lr=lr, weight_decay=weight_decay)
    
train("vit_decatt_cifar10", model, criterion, optimizer1, num_epochs, trainloader, testloader, optimizer2=optimizer2)

# Best Validation Accuracy: 69.310%
# Time to Max Val Accuracy: 17.762 mins

100%|██████████| 50/50 [18:28<00:00, 22.18s/it, Epoch=50, Train Accuracy=67.1, Training Loss=50.7, Validation Accuracy=68.8]

Best Validation Accuracy: 69.310%
Time to Max Val Accuracy: 17.762 mins


In [5]:
print(f"Total num params: {sum(p.numel() for p in model.parameters())}")
# Total num params: 4197994

Total num params: 4197994


# CIFAR100

In [3]:
image_size = 32
patch_size = 4
in_channels = 3
mlp_dim = 512
num_classes = 100
num_layers = 12
dropout = 0.1
batch_size = 256

lr = 1e-3
weight_decay = 1e-3
num_epochs = 50

trainloader, testloader = cifar100_loaders(image_size, batch_size)

### ViT Baseline

In [5]:
num_heads = 6
embed_dim = 64 * num_heads

model = VisionTransformer(
    image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout
)
model = torch.nn.DataParallel(model.cuda(), device_ids=device_ids).cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
train("vit_baseline_cifar100", model, criterion, optimizer, num_epochs, trainloader, testloader)

# Best Validation Accuracy: 40.380%
# Time to Max Val Accuracy: 24.442 mins

100%|██████████| 50/50 [24:26<00:00, 29.34s/it, Epoch=50, Train Accuracy=39, Training Loss=2.32, Validation Accuracy=40.4]  

Best Validation Accuracy: 40.380%
Time to Max Val Accuracy: 24.442 mins


In [6]:
print(f"Total num params: {sum(p.numel() for p in model.parameters())}")
# Total num params: 11983012

Total num params: 11983012


### DeCAtt Loss

In [4]:
num_heads = 3
embed_dim = 64 * num_heads

model = VisionTransformer(
    image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout
)
model = torch.nn.DataParallel(model.cuda(), device_ids=device_ids).cuda()

criterion = DecattLoss(num_heads)
optimizer1 = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
optimizer2 = torch.optim.Adam(model.module.transformer1.parameters(), lr=lr, weight_decay=weight_decay)
    
train("vit_decatt_cifar100", model, criterion, optimizer1, num_epochs, trainloader, testloader, optimizer2=optimizer2)

# Best Validation Accuracy: 40.590%
# Time to Max Val Accuracy: 18.759 mins

100%|██████████| 50/50 [19:08<00:00, 22.97s/it, Epoch=50, Train Accuracy=37.3, Training Loss=52.2, Validation Accuracy=40.2]

Best Validation Accuracy: 40.590%
Time to Max Val Accuracy: 18.759 mins


In [5]:
print(f"Total num params: {sum(p.numel() for p in model.parameters())}")
# Total num params: 4206724

Total num params: 4206724
